In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

%cd /content/drive/MyDrive/Colab\ Notebooks/MovieSentimentalAnalysis/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MovieSentimentalAnalysis


In [2]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 50.7 MB/s 


In [3]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-j488nqda
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-j488nqda
     |████████████████████████████████| 460.3 MB 6.8 kB/s 
     |████████████████████████████████| 4.0 MB 60.3 MB/s 
     |████████████████████████████████| 462 kB 58.5 MB/s 
     |████████████████████████████████| 1.2 MB 37.6 MB/s 
     |████████████████████████████████| 132 kB 38.6 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=d6c425900bc14750bcec97220b53d308f59115ddb83b8ad4dc46af991ce8ba2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ydny_ma8/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=d8ab7954262247af4f5ee726791c4988342f109b202caa1209c96fdd259c5478
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d

In [4]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
#nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Okt 
from pykospacing import Spacing

In [5]:
data = pd.read_csv('dataset/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [ ]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

In [ ]:
df = data['document'].str.lower()
df = pd.DataFrame(df)
df.columns = ['docu']
df['isalpha'] = df['docu'].str.contains('([a-z])')
df['label'] = data['label']
df[df.isalpha == True]

,docu,isalpha,label
26,initiation을 한글로 이니테이션이랜다,True,0
66,부분부분 수oo전 스토리를 떠올리게 하는 신기함 이외에는.,True,0
93,이유없음과정없음설득력없음bgm도없ㅋ음ㅋ감독맘대로 만든것같은 불친절한영화,True,0
114,뿌리깊은나무가 커피면 대왕세종은 top야,True,1
131,나레이션도 좋았고 ost도 좋았고 많은 생각을 하게 해준 드라마였다,True,1
...,...,...,...
4870,ost는 로맨스 멜로...내용은 어설픈 공포 스릴러물,True,0
4881,ebs에서 해준 영화! 잠깐 볼려고했는데 끝까지 다봄 ㅋㅋ 재밌네요,True,1
4905,재미 더럽게없음!! 1점준것중에 한국영화 죄다 10점준 매미 oo있네..,True,0
4907,"완전 비추... 집에 dvd로 있음. 야, 지루해...",True,0


In [6]:
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

#1. 전처리 (preprocess)

* 특수문자 제거 및 띄어쓰기

In [7]:
#학습 셋 전처리
data['document'] = data['document'].str.lower()
data['preprocessed'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-z ]"," ") # 특수문자를 공백으로 치환
data.sample(5)

,id,document,label,preprocessed
1046,1047,"가볍게 볼 수 있는, 그러나 결코 가볍지 않은 영화",1,가볍게 볼 수 있는 그러나 결코 가볍지 않은 영화
4453,4454,"웃기다가 무섭다가,,,, 재밌다",1,웃기다가 무섭다가 재밌다
69,70,지금 봐도 감성적이다 달달하게 느껴질 줄이야슬픈 엔딩이라 여운이 더 짙다,1,지금 봐도 감성적이다 달달하게 느껴질 줄이야슬픈 엔딩이라 여운이 더 짙다
926,927,철없는 나를 되돌아보게하는 영화 마음이 따뜻해지는영화,1,철없는 나를 되돌아보게하는 영화 마음이 따뜻해지는영화
3558,3559,김혜수의풋풋한연기90년대에시도된이명세감독의도전과감성에박수!,1,김혜수의풋풋한연기90년대에시도된이명세감독의도전과감성에박수


In [ ]:
# 인스턴스 할당
spacing = Spacing()

spaced = []
for sentence in data['preprocessed']:
    sentence = spacing(sentence)
    spaced.append(sentence)
data['spaced'] = pd.DataFrame(spaced)

* 형태소 분석 (using Okt)

In [8]:
# 인스턴스 할당
okt = Okt()

tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in data['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
data["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.


In [48]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in data['preprocessed']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa", "Foreign", "Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
data["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.


In [49]:
data.head(5)

,id,document,label,preprocessed,tokenized_stem,main_pos
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,영상 이나 음악 이 이쁘다 해도 미화 시키다 불륜 일 뿐,영상 음악 이쁘다 해도 미화 시킨 불륜 일 뿐
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,히치콕 이 이 영화 를 보다 분명 박수 를 치다,히치콕 이 영화 봤다면 분명 박수 쳤을듯
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,괜찮다 음악 영화 가 또 나오다 따뜻하다 겨울 이 되다 것 같다,괜찮은 음악 영화 또 나왔군요 따뜻한 겨울 될 것 같아요
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0,아무래도 20년도지난작품이라 지금보기는너무유치하다,아무래도 20년 도지난작품이라 지금 보기 는 너무 유치하다,아무래도 20년 지금 보기 너무 유치하다
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,지금 까지의 영화 들 이 그렇다 이 영화 역시 일본 에 대한 미화 는 여전하다,지금 영화 그랬듯 이 영화 역시 일본 대한 미화 여전하다


In [47]:
txt = '10년간 지금까지의 영화들이 그랬듯 이 영화 역시 일본에 대한 미화는 여전하다'
pos = okt.pos(txt)
print(pos)

[('10년', 'Number'), ('간', 'Foreign'), ('지금', 'Noun'), ('까지의', 'Josa'), ('영화', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('그랬듯', 'Adjective'), ('이', 'Noun'), ('영화', 'Noun'), ('역시', 'Noun'), ('일본', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('미화', 'Noun'), ('는', 'Josa'), ('여전하다', 'Adjective')]


# 2. Train & Validation - Logistic Regression

In [64]:
X = data['tokenized_stem']
y = np.array(data.label)

train, val, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

In [65]:
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)
predicted = lgs.predict(X_eval)
print(f"Accuracy: {lgs.score(X_eval, y_eval)}" )

Accuracy: 0.88


In [66]:
df = pd.DataFrame(val)
#df.insert(1, 'LR', predicted)
df.loc[:,'answer'] = y_eval
df.loc[:,'LR'] = predicted

In [67]:
df.head()

,tokenized_stem,answer,LR
3566,첨 에 좀 잼 있다 끝 에 흐지부지,0,0
4252,헉 붉다 돼지 배경 엔 천공 의 성 라퓨타 보지 않다 심하다 걱정 되다,0,0
1918,리암 니 슨 연기 몰입도 교훈,1,1
4111,개신교도 들 의 평점 조작 에 반대 한 다,0,0
1471,평점 1 점 이상준 놈 들 알바 라는 거애 내 손목 과 전 재산 건 다,0,0


# 3. Train & Validation - Random Forest

In [68]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
forest = RandomForestClassifier(n_estimators = 100, random_state = 2022)
forest = forest.fit(X_train, y_train)

predicted = forest.predict(X_eval)
print(f"Accuracy: {forest.score(X_eval, y_eval)}" )

Accuracy: 0.829


In [69]:
df.loc[:,'RF'] = predicted
df.head()

,tokenized_stem,answer,LR,RF
3566,첨 에 좀 잼 있다 끝 에 흐지부지,0,0,0
4252,헉 붉다 돼지 배경 엔 천공 의 성 라퓨타 보지 않다 심하다 걱정 되다,0,0,0
1918,리암 니 슨 연기 몰입도 교훈,1,1,1
4111,개신교도 들 의 평점 조작 에 반대 한 다,0,0,0
1471,평점 1 점 이상준 놈 들 알바 라는 거애 내 손목 과 전 재산 건 다,0,0,0


# 4. Train & Validation - Support Vector Machine

In [70]:
#Support Vector Machine
from sklearn import svm

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
clf = svm.SVC()
clf = clf.fit(X_train, y_train)

predicted = clf.predict(X_eval)
print(f"Accuracy: {clf.score(X_eval, y_eval)}" )

Accuracy: 0.894


In [71]:
df.loc[:,'SVM'] = predicted
df.head()

,tokenized_stem,answer,LR,RF,SVM
3566,첨 에 좀 잼 있다 끝 에 흐지부지,0,0,0,0
4252,헉 붉다 돼지 배경 엔 천공 의 성 라퓨타 보지 않다 심하다 걱정 되다,0,0,0,0
1918,리암 니 슨 연기 몰입도 교훈,1,1,1,1
4111,개신교도 들 의 평점 조작 에 반대 한 다,0,0,0,0
1471,평점 1 점 이상준 놈 들 알바 라는 거애 내 손목 과 전 재산 건 다,0,0,0,0


# 5. Train & Validation - Naive Bayes

In [72]:
from sklearn.naive_bayes import BernoulliNB

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=None)

X_train = vectorizer.fit_transform(train)
X_eval = vectorizer.transform(val)
NB = BernoulliNB()
NB = NB.fit(X_train, y_train)

predicted = NB.predict(X_eval)
print(f"Accuracy: {NB.score(X_eval, y_eval)}" )

Accuracy: 0.889


In [73]:
df.loc[:,'NB'] = predicted

In [75]:
df.sample(20)

,tokenized_stem,answer,LR,RF,SVM,NB
2607,비 틀다 대명사 치다 너무 전형 적 총 들 고 너무 머뭇거리다,0,0,0,0,0
4374,제 가 보다 애니메이션 영화 중 에 명작 이다,1,1,1,1,1
1367,점수 를 낮다 주다 없다 분쟁 지역 에 대한 풍자 가 아주 자다 나타나다 영화 다,1,0,0,1,1
2464,견자단 조아하다 보다 금성무 이 시 퀴 만 나오다 영화 가 ㅈㄴ 재미없다 ㅡㅡ,0,0,0,0,0
3143,덴젤 워싱턴 의 카리스마 만으로 충븐했다 멕시코시티 의 이 국적 인 배경 은 덤,1,1,1,1,1
1711,재미 만 있다 ㅋㅋㅋ 볼 만 함,1,1,1,1,1
1826,잘만 든 영화 는 아니다 재미 도 없다 평점 만 왜 이렇게 높다 모르다 ㅡㅡ,0,0,0,0,0
3939,카메라 넘다 흔들리다 아직도 울렁거리다 아쉽다,0,1,1,1,1
4977,갈피 만 제대로 잡다 충분하다 성공하다 작품,0,1,0,1,1
3214,주제곡 을 계속 들다 질리다 않다 경쾌하다 영화,1,1,1,1,1


In [46]:
df.sample(20)

,main_pos,answer,LR,RF,SVM,NB
27,개 빡치게 하는 드라마 질퍽거려너 토나올 지경,0,0,0,0,0
1572,책 비교 해서 생략 된게 많아서 아쉬웠음 근데 정말 재밋 봤음,1,1,1,1,1
4297,이기 너무나 이 기적 사람 억지스러운 이야기,0,0,1,0,1
4337,너무 너무 재미있음 디카프리오 굿 굿 굿강 추강 추,1,1,1,1,1
3375,오늘날 재판 현실 번방 기적 똑같다,1,1,1,1,1
4168,스토리 중시 여기는 나 한텐 별 점 반개 아까운 영화,0,0,0,0,0
522,롱 롱 롱 테이크 환장할 만큼 지루한 영화,0,0,0,0,0
2151,인간 버림 받은 여자 신 구원 받은 여자,1,1,0,0,0
3143,덴젤 워싱턴 카리스마 충븐했다 멕시코시티 이 국적 배경 덤,1,1,0,1,1
4666,괜히 봤다 정말 진짜 이 영화 뭥미,0,1,1,1,0


In [ ]:
out = val[val['label']!=val['answer']]
out.to_csv("wrong2.csv", index = False)